In [1]:
# Import required libraries
import csv
from dune_client.client import DuneClient

In [2]:
api_key = "WPWyWJJZEPTxvOwQmpcHbfbHoqXHaWxv"  # Replace with your Dune API key
query_id = 5301108  # Replace with your query ID

# Initialize the DuneClient
dune = DuneClient(api_key)

# Function to fetch Dune query results
def fetch_dune_query_results(query_id):
    try:
        query_result = dune.get_latest_result(query_id)
        return query_result.result.rows  # Access the rows in the response
    except Exception as e:
        print(f"Error fetching query results: {e}")
        return None

# Function to write results to CSV
def write_to_csv(data, file_name):
    try:
        # Open file in write mode
        with open(file_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            
            # Write headers (keys of the first row)
            if len(data) > 0:
                headers = data[0].keys()  # Get keys from the first row
                writer.writerow(headers)
                
                # Write rows (values)
                for row in data:
                    writer.writerow(row.values())
        print(f"Data written to {file_name}")
    except Exception as e:
        print(f"Error writing to CSV: {e}")

# Main code to fetch and save results
if __name__ == "__main__":
    result = fetch_dune_query_results(query_id)
    if result:
        file_name = "../Data/Active_Delegates_Data.csv"
        write_to_csv(result, file_name)

Data written to ../Data/Active_Delegates_Data.csv
